In [81]:
"""
Scammed link
https://stackoverflow.com/questions/61136393/keras-pretraining-of-a-cnn-model-and-after-use-it-for-a-cnn-lstm-model

"""


import tensorflow
from tensorflow.keras.layers import TimeDistributed,LSTM,Dense,Flatten,Dropout,GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50 
from tensorflow.keras import Input
from tensorflow.keras import Model 




In [76]:

def pretrained_model(seq_len=30,frames=30,channels=3,img_height=224,img_width=224,n_output=10):
    
    frames, channels, rows, columns = frames,channels,img_height,img_width

    video = Input(shape=(frames,
                         rows,
                         columns,
                         channels))
    
    cnn_base = ResNet50(input_shape=(rows,
                                  columns,
                                  channels),
                     weights="imagenet",
                     include_top=False)
    
    cnn_base.trainable = False

    cnn_out = GlobalAveragePooling2D()(cnn_base.output)
    print(cnn_out)
    
    cnn = Model(cnn_base.input,  cnn_out)
    
    encoded_frames = TimeDistributed(cnn)(video)
    
    
    encoded_sequence = LSTM(256)(encoded_frames)
    
    hidden_layer = Dense(1024, activation="relu")(encoded_sequence)
    
    outputs = Dense(n_output, activation="softmax")(hidden_layer)

    model = Model(video, outputs)

    
    return model

In [77]:
model = pretrained_model()

KerasTensor(type_spec=TensorSpec(shape=(None, 2048), dtype=tf.float32, name=None), name='global_average_pooling2d_11/Mean:0', description="created by layer 'global_average_pooling2d_11'")


In [78]:
model.summary()

Model: "model_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_53 (InputLayer)       [(None, 30, 224, 224, 3)  0         
                             ]                                   
                                                                 
 time_distributed_31 (TimeDi  (None, 30, 2048)         23587712  
 stributed)                                                      
                                                                 
 lstm_11 (LSTM)              (None, 256)               2360320   
                                                                 
 dense_18 (Dense)            (None, 1024)              263168    
                                                                 
 dense_19 (Dense)            (None, 10)                10250     
                                                                 
Total params: 26,221,450
Trainable params: 2,633,738
Non-t